# Dynamic Models - Grid Search and Model Assessment

Import Julia machine learning packages, plotting package, and file loader

In [1]:
using Flux,Statistics,Plots,MLDataUtils,DelimitedFiles
using Flux.Data: DataLoader
include("utilityfunc.jl")

evalmpe (generic function with 1 method)

Define the upper and lower bounds on the features and define the scaling and unscaling functions that execute min-max normalization on the features and target

In [2]:
#Target upper bound
conc_ub=.6
#feature lower and upper bound
i_lb=Float32[5e-7,7e-7,5.0,0.0,0.0] 
i_ub=Float32[5e-6,5e-6,30,(300/3600),conc_ub]
iscaler(x)=(x-i_lb)./(i_ub-i_lb) 
inv_iscaler(x)=x.*(i_ub.-i_lb).+i_lb
oscaler(x)= x ./ conc_ub
inv_oscaler(x)=x.*conc_ub

inv_oscaler (generic function with 1 method)

Load in the data and scale the features and output then split into training/testing/validation with 80/10/10 split

In [3]:
d = readdlm("MLFinalData_v3.csv",',',Float32)
X= iscalerbatch(d[1:5,:]);
Y= Float32.(oscaler.(d[6,:]));
(xtrg, ytrg), (xtg, ytg),(xtest,ytest) = splitobs((X, Y), at = (0.8,.1) );
datatrg=DataLoader((xtrg, ytrg));

Define the MSE loss function

In [4]:
function closs(x,y,nn)
    Flux.reset!(nn)
     Flux.mse(nn(x), y)
end

closs (generic function with 1 method)

Set training hyperparameters - 50 epochs, ADAM with a fixed learning rate of 10^-3

Then set the number of features=5,target=1

In [5]:
data_train,x_val,y_val = datatrg,xtg,ytg
opt= ADAM(1e-3)
n_epochs = 50;
nfeat =5; ntarg =1;


Execute grid search to assess 12 candidate models for the Dynamic MLP model

We vary the number of hidden layer (1-2), the activation function (tanh/ReLU), and the number of nodes per hidden layer (8,16,32)

In [6]:
nh_8,nh_16,nh_32 = 8,16,32; #candidate number of nodes
n_MLP=12 #12 canidate models
println("Beginning grid search for MLP:") 
m_list_MLP = [Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.tanh),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.tanh),Dense(nh_8, nh_8,Flux.tanh),Dense(nh_8,ntarg))
        Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.relu),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),Dense(nh_8, nh_8,Flux.relu),Dense(nh_8, nh_8,Flux.relu),Dense(nh_8,ntarg))
        Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.tanh),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.tanh),Dense(nh_16, nh_16,Flux.tanh),Dense(nh_16,ntarg))
        Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.relu),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),Dense(nh_16, nh_16,Flux.relu),Dense(nh_16, nh_16,Flux.relu),Dense(nh_16,ntarg))
        Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.tanh),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.tanh),Dense(nh_32, nh_32,Flux.tanh),Dense(nh_32,ntarg))
        Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.relu),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),Dense(nh_32, nh_32,Flux.relu),Dense(nh_32, nh_32,Flux.relu),Dense(nh_32,ntarg))
    ]
    record_MLP = Array{Any,2}(undef,2,n_MLP);
        record_MLP[1,:]=["8tanh1" "8tanh2" "8relu1" "8relu2"
    "16tanh1" "16tanh2" "16relu1" "16relu2"
        "32tanh1" "32tanh2" "32relu1" "32relu2"]
for i=1:n_MLP
    println("beginning model $i")
    iloss(x,y)=closs(x,y,m_list_MLP[i])
    Flux.reset!(m_list_MLP[i])
    ps = params(m_list_MLP[i])
    for i=1:n_epochs
        Flux.train!(iloss, ps, data_train, opt)
    end
    record_MLP[2,i]=iloss(x_val,y_val)
end
@show record_MLP

Beginning grid search for MLP:
beginning model 1
beginning model 2
beginning model 3
beginning model 4

LoadError: InterruptException:

Execute grid search to assess 6 candidate models for the Dynamic RNN model

We vary the number of hidden layer (1-2) and the number of nodes per hidden layer (8,16,32)

In [7]:
n_RNN=6
println("Beginning grid search for RNN:")
m_list_RNN = [Chain(Dense(nfeat,nh_8),RNN(nh_8, nh_8),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),RNN(nh_8, nh_8),RNN(nh_8, nh_8),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_16),RNN(nh_16, nh_16),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),RNN(nh_16, nh_16),RNN(nh_16, nh_16),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_32),RNN(nh_32, nh_32),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),RNN(nh_32, nh_32),RNN(nh_32, nh_32),Dense(nh_32,ntarg))]
        record_RNN = Array{Any,2}(undef,2,n_RNN);
        record_RNN[1,:]=["8/1" "8/2"
    "16/1" "16/2"
        "32/1" "32/2"]
for i=1:n_RNN
    println("beginning model $i")
    iloss(x,y)=closs(x,y,m_list_RNN[i])
    Flux.reset!(m_list_RNN[i])
    ps = params(m_list_RNN[i])
    for i=1:n_epochs
        Flux.train!(iloss, ps, data_train, opt)
    end
    record_RNN[2,i]=iloss(x_val,y_val)
end
@show record_RNN

Beginning grid search for RNN:
beginning model 1


LoadError: InterruptException:

Execute grid search to assess 6 candidate models for the Dynamic RNN model

We vary the number of hidden layer (1-2) and the number of nodes per hidden layer (8,16,32)

In [8]:
n_LSTM=6;
println("Beginning grid search for LSTM:")
m_list_LSTM = [Chain(Dense(nfeat,nh_8),LSTM(nh_8, nh_8),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_8),LSTM(nh_8, nh_8),LSTM(nh_8, nh_8),Dense(nh_8,ntarg))
    Chain(Dense(nfeat,nh_16),LSTM(nh_16, nh_16),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_16),LSTM(nh_16, nh_16),LSTM(nh_16, nh_16),Dense(nh_16,ntarg))
    Chain(Dense(nfeat,nh_32),LSTM(nh_32, nh_32),Dense(nh_32,ntarg))
    Chain(Dense(nfeat,nh_32),LSTM(nh_32, nh_32),LSTM(nh_32, nh_32),Dense(nh_32,ntarg))]
record_LSTM  = Array{Any,2}(undef,2,n_LSTM);
record_LSTM[1,:]=["8/1" "8/2"
    "16/1" "16/2"
        "32/1" "32/2"]
for i=1:n_LSTM
    println("beginning model $i")
    iloss(x,y)=closs(x,y,m_list_LSTM[i])
    Flux.reset!(m_list_LSTM[i])
    ps = params(m_list_LSTM[i])
    for i=1:n_epochs
        Flux.train!(iloss, ps, data_train, opt)
    end
    @show record_LSTM[2,i]=iloss(x_val,y_val)
end
@show record_LSTM 

Beginning grid search for LSTM:


LoadError: InterruptException:

A record of the original grid search run is saved in the file "record of grid search".xlsx -> it is important to note training is computationally expensive and stochastic and that any further runs may take a long time and not match the original